In [163]:
from model import Model
import torch
from torchvision.io import read_image
from torchvision.transforms import Resize, v2;
import numpy
import matplotlib.pyplot as plt
from check_orientation.pre_trained_models import create_model
import albumentations as albu

In [164]:
model = Model()
model.load_state_dict(torch.load('model_detect_corner.pth', weights_only=True))
model.freeze()

In [165]:
img = read_image("../z5950231396311_599a19af9948f46988ac81929edf00e1.jpg")
original_img = img;
img = Resize((640, 640))(img)
prediction = model(img.unsqueeze(0))

In [ ]:
print(prediction)

In [ ]:
img = img.numpy().transpose(1, 2, 0)
plt.imshow(img)
# box = []
# for i in range(4):
#     box.append(prediction[0]["boxes"][prediction[0]["labels"] == i + 1][0])
# box = numpy.stack(box, axis=0)
box = prediction[0]["boxes"][prediction[0]["scores"] > 0.8]
plt.scatter(box[:, 0] + 30, box[:, 1] + 30, label="0");
plt.scatter(box[:, 2] - 30, box[:, 3] - 30, label="1");
plt.legend()
plt.show()

In [ ]:
plt.imshow(img)
cls = prediction[0]["labels"]
for i in range(4):
    plt.scatter(box[i, 0] + 30, box[i, 1] + 30, label=f"{cls[i]}");
plt.legend()
plt.show()

In [169]:
box = box[:, 0:2]

In [ ]:
def turn_position(pivot_index, p_list):
    # Assume that pivot_index is the pivot
    def is_clockwise(vec1, vec2):
        # vec1 is previous
        # vec2 is next
        return (vec1[0] * vec2[1] - vec2[0] * vec1[1]) > 0
    
    def turn_to_vec(p1, p2):
        return [p2[0] - p1[0], p2[1] - p1[1]];

    def find_line(p1, p2):
        if(p1[0] - p2[0] == 0):
            wx = 1
            wy = 0
            b = -p1[0]
        else:
            wx = (p1[1] - p2[1]) / (p1[0] - p2[0]);
            wy = -1;
            b = p1[1] - wx * p1[0];
        return wx, wy, b;

    def find_other_twos_index(p1, p2):
        p3_id = None
        p4_id = None
        for i in range(4):
            if (p_list[i] != p1).any() and (p_list[i] != p2).any():
                if p3_id == None:
                    p3_id = i;
                else:
                    p4_id = i;

        return p3_id, p4_id
    def check_diag(p1, p2):
        p3_id, p4_id = find_other_twos_index(p1, p2);
        p3 = p_list[p3_id]
        p4 = p_list[p4_id]

        wx, wy, b = find_line(p1, p2)
        mag1 = wx * p3[0] + wy * p3[1] + b;
        mag2 = wx * p4[0] + wy * p4[1] + b;
        return (mag1 * mag2 < 0)
    
    res = 0;
    position = [None for i in range(4)];
    for other_index in range(4):
        if (p_list[other_index] != p_list[pivot_index]).any():
            if check_diag(p_list[pivot_index], p_list[other_index]) == True:
                print(other_index, pivot_index)
                position[0] = pivot_index
                position[2] = other_index;

                p3_id, p4_id = find_other_twos_index(p_list[pivot_index], p_list[other_index]);
                p3 = p_list[p3_id]
                p4 = p_list[p4_id]

                vec1 = turn_to_vec(p_list[pivot_index], p_list[other_index])
                vec2 = turn_to_vec(p_list[pivot_index], p3)

                if is_clockwise(vec1 = vec1, vec2 = vec2):
                    position[3] = p3_id;
                    position[1] = p4_id;
                else:
                    position[3] = p4_id;
                    position[1] = p3_id;
                
                return position;
print(box)
print(turn_position(0, box))
box = numpy.copy(box[turn_position(0, box), :])
print(box)

In [ ]:
original_img = original_img.numpy().transpose(1, 2, 0)
print(img.shape, original_img.shape)
print(box)
box[:, 0] = (box[:, 0] + 30) * original_img.shape[1] / img.shape[1]
box[:, 1] = (box[:, 1] + 30) * original_img.shape[0] / img.shape[0]
print(box)

plt.imshow(original_img)
for i in range(4):
    plt.scatter(box[i, 0], box[i, 1], label=f"{i}");
plt.legend()
plt.show()

In [ ]:
import cv2

turn_array = [[0, 0], [850, 0], [850, 550], [0, 550]]
img_res = None

for i in range(4):
    turn_array = turn_array[1:] + [turn_array[0]]
    points1 = numpy.array(turn_array, dtype=numpy.double)
    points2 = numpy.array(box[:, 0:2])

    h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)
    img_res = cv2.warpPerspective(original_img, h, (850, 550))
    
    plt.imshow(img_res)
    plt.show()

In [ ]:
model = create_model("swsl_resnext50_32x4d")
model.eval()
transform = albu.Compose([albu.Resize(height=224, width=224), albu.Normalize(p=1)], p=1)
prediction = model(torch.tensor(transform(image = img_res)['image']).permute(2, 0, 1).unsqueeze(0));

plt.imshow(img_res)
plt.show()

print(prediction)
turn_time = torch.argmax(prediction)


for i in range(turn_time):
    turn_array = turn_array[1:] + [turn_array[0]]
    points1 = numpy.array(turn_array, dtype=numpy.double)
    points2 = numpy.array(box[:, 0:2])

    h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)
    img_res = cv2.warpPerspective(original_img, h, (850, 550))
    
plt.imshow(img_res)
plt.show()
